#อิโมจิ
ลบอิโมจิทั้งหมด เช่น 😬, 🎉, 🚗, เป็นต้น
#ข้อความที่ลงท้ายด้วย ...
ลบข้อความที่มี ... และติดกันจนถึงช่องว่างถัดไป เช่น อ... จะถูกลบออกไป
#ตัวอักษรซ้ำ
ลดตัวอักษรที่ซ้ำกันหลายตัวให้เหลือเพียงตัวเดียว เช่น "มากกกก" -> "มาก", "เยี่ยมมม" -> "เยี่ยม"
#แฮชแท็ก
ลบข้อความที่ขึ้นต้นด้วย # เช่น #WongnaiThaiSpecialist, #อร่อยมาก
#"555"
ลบข้อความที่เป็น 555, 55555 หรือข้อความที่แสดงเสียงหัวเราะแบบนี้ออกทั้งหมด
#ข้อความที่มี "+"
แทนที่ + ระหว่างคำด้วยคำว่า "และ" เช่น ข้าว+แกง -> ข้าว และ แกง
#รูปแบบเวลา
คงรูปแบบเวลา เช่น 8:30-15:00, 18:30, 18.30 โดยไม่เปลี่ยนแปลงอะไร
#ช่วงตัวเลขที่มี -
แปลงข้อความที่มี - ระหว่างตัวเลข เช่น 100-200 เป็น 100 ถึง 200
#ข้อความติดกับตัวเลข
เพิ่มช่องว่างหน้าตัวเลขและหลังตัวเลข เช่น เสียใจทั้ง2ฝ่าย -> เสียใจทั้ง 2 ฝ่าย
ยกเว้นในกรณีที่ข้อความเป็นเวลา (18:30) หรือข้อความที่มี / (4/5)
#ข้อความที่มี /
คงรูปข้อความที่มี / เช่น 4/5, 1/2 โดยไม่เปลี่ยนแปลงอะไร
#ลบลิงก์ URL
ลบข้อความที่เป็นลิงก์ เช่น http://example.com, www.example.com
#จัดการช่องว่าง
ลบช่องว่างที่ซ้ำซ้อน และจัดรูปแบบข้อความให้สวยงาม

In [ ]:
!pip install datasets pandas

In [ ]:
!pip install pythainlp

In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.spell import correct
from datasets import load_dataset
import re

In [ ]:
import pandas as pd

# โหลดไฟล์ train.csv จาก runtime
dataset = pd.read_csv('/content/drive/MyDrive/Dataset/Combined_Data_with_Text_Length.csv')

dataset.head()

In [ ]:
# ฟังก์ชันทำความสะอาดข้อความ
def clean_text(text):
    # หากข้อความเป็นภาษาอังกฤษทั้งดุ้น ให้ลบทิ้ง
    if re.fullmatch(r'[A-Za-z0-9\s.,!?;:/()\'"%\-]+', text):
        return None

    # ลบอิโมจิ
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"  # Enclosed Characters
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub(r'', text)

    # ลบข้อความที่มี ... และติดกันจนถึงช่องว่างถัดไป
    text = re.sub(r'\S*\.{3,}', '', text)

    # ลบ "555" หรือ "55555" ออกทั้งหมด
    text = re.sub(r'\b5{3,}\b', '', text)

    # ตัดตัวอักษรซ้ำกัน เหลือเพียงตัวเดียว เช่น "มากกกก" -> "มาก"
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # ลบแฮชแท็ก เช่น "#WongnaiThaiSpecialist"
    text = re.sub(r'#\S+', '', text)

    # แทนที่ "+" ระหว่างข้อความด้วยคำว่า "และ"
    text = re.sub(r'(?<=\w)\s*\+\s*(?=\w)', ' และ ', text)

    # คงรูปเวลา เช่น "8:30-15:00" หรือ "18:00-22:00 น."
    text = re.sub(r'(\d{1,2}:\d{2}-\d{1,2}:\d{2}(?: น\.)?)', r'\1', text)

    # คงรูป 4/5 หรือข้อความที่มี "/" ไม่เปลี่ยนแปลง
    text = re.sub(r'(\d+)/(\d+)', r'\1/\2', text)

     # เปลี่ยน "100-200" เป็น "100 ถึง 200" ถ้า "-" อยู่ระหว่างตัวเลข
    text = re.sub(r'(\d+)-(\d+)', r'\1 ถึง \2', text)

    # เพิ่มช่องว่างหน้าตัวเลขและหลังตัวเลข ยกเว้นกรณีที่เป็นเวลา หรือมี /
    text = re.sub(
        r'(\D)(\d+)(\D)',
        lambda m: m.group(1) + ' ' + m.group(2) + ' ' + m.group(3)
        if ':' not in m.group(0) and '.' not in m.group(0) and '/' not in m.group(0)
        else m.group(0),
        text,
    )

    # แทนที่ \n ด้วยช่องว่าง
    text = text.replace('\n', ' ')

    # ลบลิงก์ URL
    text = re.sub(r'http\S+|www\S+', '', text)

    # ลบช่องว่างที่เกินจำเป็น
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
# ฟังก์ชันสำหรับตัดคำและแก้ไขคำผิด
def tokenize_and_correct_text(text):
    tokens = word_tokenize(text, engine='newmm')  # ตัดคำ
    corrected_tokens = [correct(word) for word in tokens]  # แก้ไขคำผิด
    return ' '.join(corrected_tokens)  # รวมคำกลับเป็นประโยค

In [ ]:
# ใช้ฟังก์ชันทำความสะอาดข้อความและตัดคำกับคอลัมน์ review_body
cleaned_reviews = []

for _, review in dataset.iterrows():
    cleaned_review = clean_text(review["review_body"])
    if cleaned_review:  # เก็บเฉพาะข้อความที่ไม่ใช่ภาษาอังกฤษทั้งดุ้น
        cleaned_reviews.append({"review_body": cleaned_review, "stars": review["stars"]})

In [ ]:
# แปลงผลลัพธ์กลับไปเป็น Dataset
from datasets import Dataset
cleaned_dataset = Dataset.from_list(cleaned_reviews)

In [ ]:
# บันทึกข้อมูลทั้งหมดเป็น CSV
import pandas as pd

In [ ]:
# แปลง Dataset เป็น DataFrame
df = pd.DataFrame(cleaned_reviews)

In [ ]:
# บันทึกเป็นไฟล์ CSV
output_file = "corrected_cleaned_data2.csv"
df.to_csv(output_file, index=False)

print(f"ข้อมูลที่ผ่านการทำความสะอาด ตัดคำ และแก้ไขคำผิด ถูกบันทึกเรียบร้อยในไฟล์: {output_file}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')